머신러닝에서 문제를 해결할 때 많은 시간이 데이터를 다루는 데 소요된다.

데이터 분석, 전처리, 파이프라인을 만드는 과정에서 약 70~80% 시간이 소비가 됨!

In [ ]:
import os
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
import numpy as np

from tensorflow.keras import preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']
           
label = [[1],[0], [1], [1], [0], [1]]

# 전처리 과정 (각 단어들을 수치화 하는 것)
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

print("수치화된 텍스트 데이터 : \n", sequences)
print("각 단어의 인덱스 : \n", word_index)
print("라벨 :", label)

Instructions for updating:
non-resource variables are not supported in the long term
수치화된 텍스트 데이터 : 
 [[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
각 단어의 인덱스 : 
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
라벨 : [[1], [0], [1], [1], [0], [1]]


In [ ]:
# Sequences 에 있는 단어들을 하나씩 출력해보는 과정

dataset = tf.data.Dataset.from_tensor_slices((sequences, label)) # sequences 와 label을 묶어서 조각으로 만들어준다.
iterator = dataset.make_one_shot_iterator() # 데이터를 하나씩 사용할 수 있게 만드는 함수
next_data = iterator.get_next() # 데이터가 하나씩 나오게 되는 구조

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


In [ ]:
# 배치 크기를 정의한 후 데이터가 배치크기 만큼 한번에 사용할 수 있게 하기
BATCH_SIZE=3

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.batch(BATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try :
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([[ 4,  1,  5,  6],
       [ 7,  1,  8,  9],
       [10,  2,  3, 11]], dtype=int32), array([[1],
       [0],
       [1]], dtype=int32))
(array([[12,  2,  3, 13],
       [14,  1, 15, 16],
       [17, 18, 19, 20]], dtype=int32), array([[1],
       [0],
       [1]], dtype=int32))


셔플 처리는 이후 모델의 성능에 크게 기여하기 때문에 매우 중요하다!!!!!

#### Shuffle

In [ ]:
# 데이터를 섞어서 출력하도록 하기
# dataset.shuffle
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.shuffle(len(sequences))
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


#### Repeat

In [ ]:
# 모델에 몇번을 학습시킬 것인가를 정해주는 것!! epoch!!!
# dataset.repeat
EPOCH=2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.repeat(EPOCH) # EPOCH 수만큼 반복시킨다!
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()
with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


#### Mapping

- 딕셔너리 구조로 활용된다.
- 입력값은 하나의 딕셔너리로 묶어야 한다. 아래의 코드는 하나의 입력값만 사용했기 때문에 딕셔너리 안에 하나의 값만 존재하는데, 만약 두개 이상의 입력값이 존재한다면  아래와 같이 매핑함수를 정의해야한다

    <python>python code :
      def mapping_fn(X1, X2, Y=None):
        input = {'x1':X1, 'x2':X2}
        label = Y
        return input, label
    </python>

In [ ]:

def mapping_fn(X, Y=None):
  input= {'x' : X}
  label= Y
  return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

({'x': array([4, 1, 5, 6], dtype=int32)}, array([1], dtype=int32))
({'x': array([7, 1, 8, 9], dtype=int32)}, array([0], dtype=int32))
({'x': array([10,  2,  3, 11], dtype=int32)}, array([1], dtype=int32))
({'x': array([12,  2,  3, 13], dtype=int32)}, array([1], dtype=int32))
({'x': array([14,  1, 15, 16], dtype=int32)}, array([0], dtype=int32))
({'x': array([17, 18, 19, 20], dtype=int32)}, array([1], dtype=int32))


배치, 셔플, 반복, 매핑 과정을 한번에 하기
- dataset 에 함수를 중첩해서 적용하면 된다!!!

In [ ]:
BATCH_SIZE =2
EPOCH =2

def mapping_fn(X, Y=None):
  input = {'x':X}
  label = Y
  return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
dataset = dataset.shuffle(len(sequences))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat(EPOCH)

iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

({'x': array([[ 7,  1,  8,  9],
       [17, 18, 19, 20]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[10,  2,  3, 11],
       [ 4,  1,  5,  6]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[12,  2,  3, 13],
       [14,  1, 15, 16]], dtype=int32)}, array([[1],
       [0]], dtype=int32))
({'x': array([[ 7,  1,  8,  9],
       [10,  2,  3, 11]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[12,  2,  3, 13],
       [ 4,  1,  5,  6]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[14,  1, 15, 16],
       [17, 18, 19, 20]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
